### Data shapes Flow 

```py
input: (B, T)

C -> emb dim

1. emb: (input) -> (B, T, C) : emb + pos_emb
2. Block:
    MHA: (B, T, C) -> (B, T, H, C') -> (B, H, T, C') -> mha -> (B T H C') -> (B T C) 
    Feedfwd: (B T C) -> C -> expands in hidden dim then comes to -> (B T C)

3. Linear: (B T C) -> (B T vocab_size)
4. calc Loss fn: 
    calc loss b/w pred (B * T, vocab) vs target (B * T,)
    -> (B T vocab) -> (B * T, vocab) -> softmax -> [ (BT, vocab), (BT, ) ] -> (BT, -log(p_true_class) ) -> mean(BT, 1) -> single scaler loss and backprob
        loss = -prob[np.arange(n), ys].log().mean()
```
e.g.
```py
    logits_flat =
    [[2.0, 1.0, 0.1],    # target: 0
    [1.0, 3.0, 0.2],    # target: 1
    [0.5, 2.0, 0.3],    # target: 1
    [1.5, 0.2, 2.0]]    # target: 2

    targets_flat = [0, 1, 1, 2]

    loss = log_softmax(logits)  # then picks the log-prob for the correct class
    loss = -log(p_true_class)
    loss = mean over all examples

    Input: [2.0, 1.0, 0.1]
    Softmax = exp(x_i) / sum(exp(x_j))
            = [e^2, e^1, e^0.1] / sum(...)
            ≈ [7.389, 2.718, 1.105] / (7.389+2.718+1.105)
            ≈ [0.659, 0.242, 0.098]

    Log-softmax ≈ [log(0.659), log(0.242), log(0.098)] 
                ≈ [-0.417, -1.417, -2.325]

    Loss_1 = -log(0.659) ≈ 0.417
```
| Row | Logits           | Softmax probs          | Target | Loss                |
| --- | ---------------- | ---------------------- | ------ | ------------------- |
| 1   | \[2.0, 1.0, 0.1] | \[0.659, 0.242, 0.098] | 0      | -log(0.659) ≈ 0.417 |
| 2   | \[1.0, 3.0, 0.2] | \[0.113, 0.836, 0.051] | 1      | -log(0.836) ≈ 0.179 |
| 3   | \[0.5, 2.0, 0.3] | \[0.211, 0.627, 0.162] | 1      | -log(0.627) ≈ 0.466 |
| 4   | \[1.5, 0.2, 2.0] | \[0.331, 0.097, 0.571] | 2      | -log(0.571) ≈ 0.560 |

